<a href="https://colab.research.google.com/github/francescogrillea/AnomalyDetectionTFLite/blob/main/AnomalyDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!cat ./drive/MyDrive/Colab*/anomaly_detection_data/res*

In [ ]:
!rm ./drive/MyDrive/Colab*/anomaly_detection_data/res*

# Import e Costanti

In [51]:
import os
import tensorflow as tf
import numpy as np
from tensorflow import keras
import time
import logging
import yaml

cpu = os.popen('lscpu |grep \'Model name\'').read()
ram_free = os.popen('free -h --si | awk  \'/Mem:/{print $4}\'').read()
ram_total = os.popen('free -h --si | awk  \'/Mem:/{print $2}\'').read()

logging.basicConfig(filename="./drive/MyDrive/Colab Notebooks/anomaly_detection_data/results.log", level=logging.INFO, format='%(message)s')
logging.info('CPU\n\t{}'.format(cpu))
logging.info('RAM\n\t{} / {} free'.format(ram_free[:-1], ram_total[:-1]))

In [105]:
CONFIG_PATH = "./drive/MyDrive/Colab Notebooks/anomaly_detection_data/config.yaml"
with open(CONFIG_PATH, "r") as f:
    config = yaml.safe_load(f)

PROJECT_PATH = config['PROJECT_PATH']
ARCHITECTURE_TYPE = config['ARCHITECTURE_TYPE']
CHANNEL_NAME = config['CHANNEL_NAME']

DATASET_PATH = PROJECT_PATH+'Dataset/'
TF_MODELS_PATH = PROJECT_PATH+'TF_Models/'+ARCHITECTURE_TYPE+'/'
TFLite_MODELS_PATH = PROJECT_PATH+'TFLite_Models/'+ARCHITECTURE_TYPE+'/'
RESULT_FILE_PATH = TFLite_MODELS_PATH+'results.yaml'

try:
  with open(RESULT_FILE_PATH, "r") as f:
    results = yaml.safe_load(f)
except FileNotFoundError:
    results = {}

if CHANNEL_NAME is None:
  channels = [f for f in os.listdir(TF_MODELS_PATH)]
  for c in channels:
    b = c[:-3]
    results[b] = {}
else:
  if not CHANNEL_NAME in results:
    results[CHANNEL_NAME] = {}


if not '_CPU' in results:
  results['_CPU'] = cpu.strip()[21:]
if not '_RAM' in results:
  results['_RAM'] = ram_total.strip()


logging.info('---------------------------------')
logging.info('{}\nArchitecture: {}\nChannel: {}'.format(time.ctime(), ARCHITECTURE_TYPE, CHANNEL_NAME))

def load_seed(channel_name):
  with open(TF_MODELS_PATH+'seeds.log', 'r') as f:
    for row in f.readlines():
      if row.startswith(CHANNEL_NAME):
        return row[4:]

#load .h5 model
def load_tf_model(channel_name):
  if not channel_name.endswith('.h5'):
    channel_name = channel_name+'.h5'
  tf_model = keras.models.load_model(TF_MODELS_PATH+channel_name)
  return tf_model

#load dataset
def load_dataset(channel_name):
  if not channel_name.endswith('.npy'):
    channel_name = channel_name+'.npy'
  x = np.array([np.load(DATASET_PATH+'test/'+channel_name)])
  y = np.array([])
  return x,y

def update_results():
  with open(RESULT_FILE_PATH, 'w') as f:
    yaml.dump(results, f)

In [99]:
update_results()

# Prediction su TF Model

In [106]:
#load dataset
tf_input_data, tf_output_data = load_dataset(CHANNEL_NAME)

#load model
model = load_tf_model(CHANNEL_NAME)

start_time = time.time()
tf_output_data = model.predict(tf_input_data)
delta_time = time.time()-start_time

results[CHANNEL_NAME]['TF Model prediction time'] = delta_time
results[CHANNEL_NAME]['TF Model predictions'] = tf_output_data[0].tolist()

logging.info('TensorFlow prediction time: {}s'.format(delta_time))
print('TensorFlow prediction time: {}s'.format(delta_time))

TensorFlow prediction time: 0.7707931995391846s


# Conversione dei modelli da .h5 a .tflite

In [107]:
channels = [f for f in os.listdir(TF_MODELS_PATH)]

if not CHANNEL_NAME is None:
  channels = [c for c in channels if c.startswith(CHANNEL_NAME)]

n_channels = len(channels) 
counter = 0

for channel in channels:
  counter += 1
  tflite_model_path = TFLite_MODELS_PATH+channel[:-3]+'.tflite'

  if not os.path.isfile(tflite_model_path):
    logging.info('Convert {} model from TensorFlow to TensorFlow Lite'.format(CHANNEL_NAME))
    
    start_time = time.time()
    #load TF Model
    tf_model = load_tf_model(channel)
    #STATS- TF model size (in Kb)
    tf_model_size = int(os.path.getsize(TF_MODELS_PATH+channel) / 1024)
    results[channel[:-3]]['TF Model size'] = tf_model_size

    #convert model to TensorFlow Lite
    converter = tf.lite.TFLiteConverter.from_keras_model(tf_model)
    converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
    tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
  ]
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    #WARNING:absl:Found untraced functions such as lstm_cell -> e' un warning che dipende da come è stato salvato il modello, tuttavia non sembra esserci una precisa soluzione
    tflite_model = converter.convert()

    delta_time = time.time() - start_time

    #generate TFLite model name and path
    tflite_model_filename = channel[:-3]+'.tflite'
    TFLite_MODEL_FILE = TFLite_MODELS_PATH+tflite_model_filename

    #save TFLite model
    with open(TFLite_MODEL_FILE, 'wb') as f:
      f.write(tflite_model)
    #STATS- .tflite model size (in Kb)
    tflite_model_size = int(os.path.getsize(TFLite_MODEL_FILE) / 1024)
    results[channel[:-3]]['TFLite Model size'] = tflite_model_size
    results[channel[:-3]]['TFLite Model conversion time'] = delta_time

    logging.info('Model '+channel[:-3]+' converted (from '+str(tf_model_size)+'Kb to '+str(tflite_model_size)+'Kb) in '+str(delta_time)+'s')
    print(str(counter)+'/'+str(n_channels)+'- '+'Model '+channel[:-3]+' converted (from '+str(tf_model_size)+'Kb to '+str(tflite_model_size)+'Kb) in '+str(delta_time)+'s')

INFO:tensorflow:Assets written to: /tmp/tmpw8rvi_lv/assets
1/1- Model A-2 converted (from 330Kb to 38Kb) in 10.760563611984253s


# Prediction su TFLite

In [108]:
#load dataset
x_data, tflite_output_data = load_dataset(CHANNEL_NAME)

interpreter = tf.lite.Interpreter(model_path=TFLite_MODELS_PATH+CHANNEL_NAME+'.tflite')
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

#preprocessing input data
input_shape = input_details[0]['shape']     #[1, 1, 25]
output_shape = output_details[0]['shape']   #[1, 10]


interpreter.resize_tensor_input(input_details[0]['index'], [1, x_data.shape[1],25])
interpreter.allocate_tensors()

input_data = np.array(x_data, dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], input_data)

start_time = time.time()
#run inference
interpreter.invoke()
tflite_output_data  = interpreter.get_tensor(output_details[0]['index'])
delta_time = time.time() - start_time

results[CHANNEL_NAME]['TFLite Model prediction time'] = delta_time
results[CHANNEL_NAME]['TFLite Model predictions'] = tflite_output_data[0].tolist()
results[CHANNEL_NAME]['Distance'] = (tf_output_data[0] - tflite_output_data[0]).tolist()

logging.info('TensorFlowLite prediction time: {}s'.format(delta_time))
print('TensorFlowLite prediction time: {}s'.format(delta_time))
update_results()

# Compare TF predictions with TFLite predictions
try:
  for y_tf, y_tflite in zip(tf_output_data[0], tflite_output_data[0]):
    logging.info('{}\t{}\t{}'.format(y_tf, y_tflite, abs(y_tf-y_tflite)))
    print('{}\t{}\t{}'.format(y_tf, y_tflite, abs(y_tf-y_tflite)))

except NameError:
  for val in tflite_output_data[0]:
    print(val)

TensorFlowLite prediction time: 1.5116493701934814s
-0.05996561795473099	-0.05363877862691879	0.006326839327812195
-0.0331600159406662	-0.027672262862324715	0.005487753078341484
-0.008841438218951225	-0.005508603528141975	0.00333283469080925
-0.014330176636576653	-0.01231645792722702	0.0020137187093496323
-0.053852785378694534	-0.053580496460199356	0.0002722889184951782
-0.11776037514209747	-0.11875312775373459	0.0009927526116371155
-0.1684092879295349	-0.16792014241218567	0.0004891455173492432
-0.23674480617046356	-0.23825813829898834	0.0015133321285247803
-0.3546895384788513	-0.35853102803230286	0.003841489553451538
-0.47840437293052673	-0.4867083728313446	0.008303999900817871
